In [1]:
from PIL import Image
import os
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re

In [2]:
from keras.preprocessing.image import ImageDataGenerator, img_to_array, array_to_img
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense, BatchNormalization
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras import backend as K
from keras.optimizers import RMSprop
import keras

Using TensorFlow backend.


In [3]:
# allocates GPU memory based on runtime allocations. doesn't releases memory because of potential memory fragmentation
# without this option, getting images will not work
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [21]:
# options
img_width, img_height = 100, 100
batch_size = 16
sample_size = 100
epochs = 10

# number of classification outputs
num_classes=2

data_dir = 'data/food'
resized_dir = '/resized'
original_dir = '/og'

In [5]:
# resizes images from directory into new subdirectory
def resize(folder, og_dir, new_dir, fileName, width, height):
    filePath = os.path.join(folder, fileName)
    im = Image.open(filePath)
    newIm = im.resize((int(width), int(height)))
    directory = re.sub(og_dir, new_dir, folder, count=1) 
    
    # create directory if it doesn't exist
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    # create new file only if it doesn't exist
    if not os.path.exists(directory + '/' + fileName):
        newIm.save(directory + '/' + fileName)

# finds images in directory
def bulkResize(imageFolder, original_dir, new_dir, width, height):
    imgExts = ["png", "bmp", "jpg"]
    for path, dirs, files in os.walk(imageFolder + original_dir):
        for fileName in files:
            ext = fileName[-3:].lower()
            if ext not in imgExts:
                continue

            resize(path, original_dir, new_dir, fileName, width, height)

bulkResize(data_dir, original_dir, resized_dir, img_width, img_height)

In [6]:
# # convert image to array and add to df
# def loadImgToArray(path, fileName):
#     im = Image.open(path+'/'+fileName)
#     ar = img_to_array(im)
#     ar = ar/255.0
#     reshape = ar.reshape(img_width*img_height, 3)
#     batch_array = np.concatenate(batch_array, np.array(reshape.shape[0]))
# #     batch_label.append()
#     remove = re.sub(data_dir+resized_dir, '', path)
#     remove = re.sub(r'\\', '', remove)
#     batch_label = np.concatenate(batch_label, np.array(remove))


# # convert to arrays
# def bulkConvert(imageFolder):
#     imgExts = ["png", "bmp", "jpg"]
#     for path, dirs, files in os.walk(imageFolder):
#         for fileName in files:
#             ext = fileName[-3:].lower()
#             if ext not in imgExts:
#                 continue

#             loadImgToArray(path, fileName)
    
    
# batch_array = np.empty(batch_size)
# batch_label = np.empty(batch_size)
# bulkConvert(data_dir + resized_dir)

# feat_cols = [ 'pixel '+str(i+1) for i in range(img_width*img_height)]
# df = pd.DataFrame(batch_array, columns=feat_cols)


In [7]:
# type(batch_array)

In [6]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [22]:
datagen = ImageDataGenerator(horizontal_flip=True)

train_generator = datagen.flow_from_directory(
directory = data_dir+resized_dir,
target_size = (img_width, img_height),
batch_size = batch_size,
class_mode = 'categorical')

Found 100 images belonging to 2 classes.


In [23]:
# CNN model with batch normalization
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(num_classes, activation='softmax'))

model.summary()
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit_generator(
    train_generator,
    steps_per_epoch= sample_size // batch_size,
    epochs=epochs
)



prediction = model.predict_generator(train_generator, verbose=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 98, 98, 32)        896       
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 96, 96, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 48, 48, 64)        0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 48, 48, 64)        0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 147456)            0         
_________________________________________________________________
dense_11 (Dense)             (None, 128)               18874496  
_________________________________________________________________
dropout_12 (Dropout)         (None, 128)               0         
__________

In [24]:
prediction

array([[0.86027575, 0.13972424],
       [0.93687767, 0.0631223 ],
       [0.9276148 , 0.07238524],
       [0.3128816 , 0.6871184 ],
       [0.9335442 , 0.06645579],
       [0.73841816, 0.2615819 ],
       [0.8585424 , 0.14145768],
       [0.9803767 , 0.01962325],
       [0.96327025, 0.03672975],
       [0.7545611 , 0.24543881],
       [0.956774  , 0.04322594],
       [0.65553147, 0.3444685 ],
       [0.92528325, 0.07471675],
       [0.6972712 , 0.30272877],
       [0.87768614, 0.12231386],
       [0.5677709 , 0.43222913],
       [0.965712  , 0.03428799],
       [0.9619254 , 0.0380746 ],
       [0.9415436 , 0.05845647],
       [0.9015077 , 0.09849238],
       [0.88028425, 0.11971575],
       [0.7736346 , 0.22636537],
       [0.6785167 , 0.32148337],
       [0.7531976 , 0.24680237],
       [0.98974377, 0.01025627],
       [0.99382186, 0.00617808],
       [0.72819495, 0.271805  ],
       [0.78353   , 0.21646994],
       [0.9355596 , 0.0644404 ],
       [0.4491087 , 0.5508913 ],
       [0.